# Water Cloud Model (WCM) and Dubois Model Implementation

This notebook implements and evaluates the Water Cloud Model (Prevot, 1993) and Dubois Model for radar backscatter estimation using Sentinel-1 data.

The main objectives are:
1. Implement the WCM model for VV and VH polarizations
2. Optimize model parameters using different optimization techniques
3. Evaluate model performance using R² score
4. Implement ML model stacking to improve predictions
5. Compare with the Dubois model

## Import Libraries

In [ ]:
# Data manipulation and analysis
import pandas as pd
import numpy as np

# Machine learning and optimization
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Optimization methods
from scipy.optimize import least_squares, basinhopping, differential_evolution, minimize
import scipy.constants

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Utilities
from IPython.display import clear_output
from functools import partial
import warnings

# Ignore warnings for cleaner output
warnings.filterwarnings('ignore')

## Define Constants

These constants are used in the radar backscatter models:
- f: Frequency (5.4 GHz for Sentinel-1)
- c: Speed of light
- wavelength: Wavelength calculated from frequency
- k: Wave number
- s: Surface roughness parameter
- sbl: Another surface roughness parameter
- theta: Incidence angle in degrees
- theta_rad: Incidence angle in radians

In [ ]:
# Radar parameters
f = 5.4e9  # Frequency in Hz (5.4 GHz for Sentinel-1)
c = scipy.constants.c  # Speed of light
wavelength = c / f  # Wavelength in meters
k = 2 * np.pi * f / c  # Wave number

# Surface parameters
s = 0.0097  # Surface roughness parameter
sbl = 0.013  # Another surface roughness parameter

# Incidence angle
theta = 40  # Degrees
theta_rad = np.deg2rad(theta)  # Radians

# Display constants for reference
print(f"Frequency: {f/1e9} GHz")
print(f"Wavelength: {wavelength:.4f} m")
print(f"Incidence angle: {theta}° ({theta_rad:.4f} rad)")
print(f"Surface roughness (s): {s}")
print(f"Surface roughness (sbl): {sbl}")

## Load and Preprocess Data

We load the dataset containing:
- VV: Vertical-Vertical polarization backscatter (dB)
- VH: Vertical-Horizontal polarization backscatter (dB)
- SM: Soil Moisture
- LAI: Leaf Area Index

The data is trimmed to remove outliers based on standard deviation.

In [ ]:
# Load the dataset
data = pd.read_csv(r"../Data/Merged_Data.csv")

# Display basic information about the dataset
print("Dataset shape:", data.shape)
print("\nFirst 5 rows:")
display(data.head())

print("\nBasic statistics:")
display(data.describe())

In [ ]:
# Visualize the distributions of key variables
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

sns.histplot(data['VV'], kde=True, ax=axes[0, 0])
axes[0, 0].set_title('VV Distribution')

sns.histplot(data['VH'], kde=True, ax=axes[0, 1])
axes[0, 1].set_title('VH Distribution')

sns.histplot(data['SM'], kde=True, ax=axes[1, 0])
axes[1, 0].set_title('Soil Moisture Distribution')

sns.histplot(data['LAI'], kde=True, ax=axes[1, 1])
axes[1, 1].set_title('Leaf Area Index Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Visualize relationships between variables
plt.figure(figsize=(12, 10))
sns.pairplot(data[['VV', 'VH', 'SM', 'LAI']])
plt.suptitle('Pairwise Relationships Between Variables', y=1.02)
plt.show()

In [ ]:
# Function to trim outliers based on standard deviation
def trim_outliers(df, column, n_std=4):
    """Trim outliers from a dataframe based on standard deviation.
    
    Args:
        df: DataFrame containing the data
        column: Column name to trim outliers from
        n_std: Number of standard deviations to use as threshold
        
    Returns:
        DataFrame with outliers removed
    """
    mean = df[column].mean()
    std = df[column].std()
    return df[(df[column] >= mean - n_std * std) & (df[column] <= mean + n_std * std)]

# Trim outliers for VV and VH
trim_number = 4  # Number of standard deviations to use as threshold
data_trimmed = data.copy()
data_trimmed = trim_outliers(data_trimmed, 'VV', trim_number)
data_trimmed = trim_outliers(data_trimmed, 'VH', trim_number)

print(f"Original data shape: {data.shape}")
print(f"Trimmed data shape: {data_trimmed.shape}")
print(f"Removed {data.shape[0] - data_trimmed.shape[0]} rows ({(data.shape[0] - data_trimmed.shape[0])/data.shape[0]*100:.2f}%)")

In [ ]:
# Extract data for modeling
segment = 0  # Segment of data to use (0 means first segment)
segment_size = 41  # Size of each segment

# If segment is 0, use all data instead of slicing
if segment == 0 and segment_size >= len(data_trimmed):
    VV_dB = data_trimmed['VV'].values
    VH_dB = data_trimmed['VH'].values
    SM = data_trimmed['SM'].values
    LAI = data_trimmed['LAI'].values
else:
    # Extract segment
    start_idx = segment * segment_size
    end_idx = min(start_idx + segment_size, len(data_trimmed))
    
    VV_dB = data_trimmed['VV'].values[start_idx:end_idx]
    VH_dB = data_trimmed['VH'].values[start_idx:end_idx]
    SM = data_trimmed['SM'].values[start_idx:end_idx]
    LAI = data_trimmed['LAI'].values[start_idx:end_idx]

# Convert from dB to linear scale
VV_linear = 10**(VV_dB / 10)
VH_linear = 10**(VH_dB / 10)

print(f"Data shape after segmentation: {len(VV_dB)} samples")

## Water Cloud Model Implementation

The Water Cloud Model (WCM) by Prevot (1993) models the radar backscatter as a combination of:
1. Direct vegetation contribution
2. Attenuated soil contribution

The model has 5 parameters (P1-P5) that need to be optimized.

In [ ]:
def wcm_1993_sigma_0(P1, P2, P3, P4, P5, L, S):
    """Prevot 1993 Water Cloud Model implementation.
    
    Args:
        P1-P5: Model parameters to be optimized
        L: Leaf Area Index
        S: Soil Moisture
        
    Returns:
        Estimated radar backscatter
    """
    # Calculate two-way transmissivity
    t2 = np.exp(-2 * P2 * L / np.cos(theta_rad))
    
    # Vegetation contribution
    # Note: The (1-t2) factor is omitted as it was found to reduce R² score
    sigma_veg = P1 * np.power(L, P5) * np.cos(theta_rad)
    
    # Soil contribution
    sigma_soil = P3 + P4 * S
    
    # Total backscatter
    return sigma_veg + (t2 * sigma_soil)

## Optimization Functions

We implement several optimization methods to find the best parameters for the WCM model:
1. Least Squares (LS)
2. Differential Evolution (DE) with local optimization

Note: Basin-hopping was found to be unstable for this function and is commented out.

In [ ]:
def optimize_wcm_1993_sigma_0_ls(polarization, L, S):
    """Optimize WCM parameters using Least Squares method.
    
    Args:
        polarization: VV or VH backscatter values
        L: Leaf Area Index values
        S: Soil Moisture values
        
    Returns:
        Optimized parameters
    """
    def residuals(params):
        predicted = wcm_1993_sigma_0(*params, L, S)
        residuals = predicted - polarization
        # Handle numerical instabilities
        if not np.all(np.isfinite(residuals)):
            return np.ones_like(residuals) * np.inf 
        return residuals
    
    # Initial parameter guess
    initial_guess = [0.1, 1.3, 1.2, 0.9, 0.8]
    
    # Optimize using least squares with robust loss function
    result = least_squares(
        residuals, 
        initial_guess, 
        method='trf',  # Trust Region Reflective algorithm
        loss='soft_l1',  # Robust loss function
        max_nfev=10000  # Maximum number of function evaluations
    )
    
    return result.x

In [ ]:
def optimize_wcm_1993_sigma_0_de_hybrid(polarization, L, S):
    """Optimize WCM parameters using Differential Evolution followed by local optimization.
    
    This hybrid approach first uses a global optimizer (Differential Evolution) to find
    a good starting point, then refines it with a local optimizer (L-BFGS-B).
    
    Args:
        polarization: VV or VH backscatter values
        L: Leaf Area Index values
        S: Soil Moisture values
        
    Returns:
        Optimized parameters
    """
    def objective(params):
        predicted = wcm_1993_sigma_0(*params, L, S)
        residuals = predicted - polarization
        # Use sum of squared residuals as objective
        return np.sum(residuals**2)  

    # Parameter bounds
    bounds = [(-10, 10), (-10, 10), (-10, 10), (-10, 10), (0.01, 10)]  
    
    # First stage: global optimization with Differential Evolution
    result_de = differential_evolution(
        objective,
        bounds,
        maxiter=1000,  # Maximum number of generations
        popsize=20,    # Population size
        tol=1e-6,      # Convergence tolerance
        mutation=(0.5, 1.0),  # Mutation constant
        recombination=0.7,    # Recombination constant
        seed=42              # For reproducibility
    )
    
    # Second stage: local optimization starting from DE result
    result_local = minimize(
        objective, 
        result_de.x, 
        method='L-BFGS-B',  # Limited-memory BFGS with bounds
        bounds=bounds
    )
    
    return result_local.x

In [ ]:
def wcm_1993_validate_optimizer(optimizer_func, data, plot=True):
    """Validate WCM optimizer performance for both VV and VH polarizations.
    
    Args:
        optimizer_func: Function to optimize WCM parameters
        data: DataFrame containing the data
        plot: Whether to plot the results
        
    Returns:
        Tuple of (predicted_VV, predicted_VH)
    """
    # Optimize for VV polarization
    params_VV = optimizer_func(VV_dB, LAI, SM)
    predicted_VV = wcm_1993_sigma_0(*params_VV, LAI, SM)
    r2_VV = r2_score(VV_dB, predicted_VV)
    rmse_VV = np.sqrt(mean_squared_error(VV_dB, predicted_VV))

    # Optimize for VH polarization
    params_VH = optimizer_func(VH_dB, LAI, SM)
    predicted_VH = wcm_1993_sigma_0(*params_VH, LAI, SM)
    r2_VH = r2_score(VH_dB, predicted_VH)
    rmse_VH = np.sqrt(mean_squared_error(VH_dB, predicted_VH))

    # Clear previous output and display results
    clear_output()
    print(f"WCM Model Parameters (VV): {params_VV}")
    print(f"WCM Model Parameters (VH): {params_VH}")
    print(f"R² Score for VV: {r2_VV:.4f}")
    print(f"RMSE for VV: {rmse_VV:.4f} dB")
    print(f"R² Score for VH: {r2_VH:.4f}")
    print(f"RMSE for VH: {rmse_VH:.4f} dB")
    
    # Plot results if requested
    if plot:
        fig, axes = plt.subplots(1, 2, figsize=(16, 6))
        
        # VV plot
        axes[0].scatter(VV_dB, predicted_VV, alpha=0.7)
        min_val = min(VV_dB.min(), predicted_VV.min())
        max_val = max(VV_dB.max(), predicted_VV.max())
        axes[0].plot([min_val, max_val], [min_val, max_val], 'r--')
        axes[0].set_xlabel('Observed VV (dB)')
        axes[0].set_ylabel('Predicted VV (dB)')
        axes[0].set_title(f'VV Polarization (R² = {r2_VV:.4f}, RMSE = {rmse_VV:.4f} dB)')
        axes[0].grid(True, alpha=0.3)
        
        # VH plot
        axes[1].scatter(VH_dB, predicted_VH, alpha=0.7)
        min_val = min(VH_dB.min(), predicted_VH.min())
        max_val = max(VH_dB.max(), predicted_VH.max())
        axes[1].plot([min_val, max_val], [min_val, max_val], 'r--')
        axes[1].set_xlabel('Observed VH (dB)')
        axes[1].set_ylabel('Predicted VH (dB)')
        axes[1].set_title(f'VH Polarization (R² = {r2_VH:.4f}, RMSE = {rmse_VH:.4f} dB)')
        axes[1].grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
    
    return (predicted_VV, predicted_VH)

## Evaluate WCM with Different Optimizers

In [ ]:
# Evaluate WCM with Least Squares optimizer
print("Evaluating WCM with Least Squares optimizer...")
pred_VV_ls, pred_VH_ls = wcm_1993_validate_optimizer(optimize_wcm_1993_sigma_0_ls, data_trimmed)

In [ ]:
# Evaluate WCM with Differential Evolution + Local optimizer
print("Evaluating WCM with Differential Evolution + Local optimizer...")
pred_VV_de, pred_VH_de = wcm_1993_validate_optimizer(optimize_wcm_1993_sigma_0_de_hybrid, data_trimmed)

## ML Model Stacking

We use the WCM predictions as features for machine learning models to improve the overall prediction accuracy.

In [ ]:
# Prepare data for ML model stacking
df = pd.read_csv(r"../Data/Merged_Data.csv")

# Get predictions from the best WCM model (DE hybrid)
pred_VV, pred_VH = pred_VV_de, pred_VH_de

# Create feature matrix
X = df[['LAI', 'SM']].copy()
X['pred_VV'] = pred_VV
X['pred_VH'] = pred_VH

# Create target variables (residuals between observed and WCM predictions)
y_vv = df['VV'] - X['pred_VV']
y_vh = df['VH'] - X['pred_VH']

# Split into train/test sets (80% train, 20% test)
X_train, X_test, y_vv_train, y_vv_test = train_test_split(X, y_vv, test_size=0.2, random_state=42)
_, _, y_vh_train, y_vh_test = train_test_split(X, y_vh, test_size=0.2, random_state=42)

print(f"Training set shape: {X_train.shape}")
print(f"Test set shape: {X_test.shape}")

### Random Forest Model